<style type="text/css">

body{ /* Normal  */
/*    font-family: Lato, sans-serif;  
      font-family: Mukta, sans-serif; 
      font-family: 'Nunito Sans', sans-serif;
      font-family: Karla, sans-serif;  */
      font-family: 'Merriweather Sans', sans-serif; 
      font-size: 18px;
  }

h1.title {
  font-size: 38px;
  color: DarkRed;
}

h1 { /* Header 1 */
  font-size: 28px;
  color: DarkBlue;
}

h2 { /* Header 2 */
    font-size: 22px;
  color: DarkBlue;
}

h3 { /* Header 3 */
  font-size: 18px;
  color: DarkBlue;
}

code.r{ /* Code block */
    font-family: Mukta, sans-serif; 
    font-weight: 600;  
    font-size: 16px;
}

/* pre { /* Code block - determines code spacing between lines */
    font-size: 16px;
} */
</style>


# MPA 5830 - Module 03 (Fall 2021)

In the previous week we learned how to clean and prepare data with several {dplyr} functions -- `select()`, `filter()`, `summarise()`, `mutate()`, `group_by()`, `case_when()`, and so on. In this module we will tackle other challenging problems and lean on another package, `{tidyr}` to do so. 

# {tidyr}
This library seems to have only four core functions, `separate`, `unite()`, `pivot_longer`, and `pivot_wider()` but each of them packs a wallop.  

## separate()
You will, at times, end up with columns that contain multiple pieces of information, all mashed up into some alphanumeric string or sequence of numbers. `separate()` allows us to split this mashed up column into specific pieces. For example, here are some data from the Census Bureau: 

In [ ]:
library(tidyverse)

read_csv(
  "https://www2.census.gov/programs-surveys/popest/datasets/2010-2018/metro/totals/cbsa-est2018-alldata.csv"
  ) -> cbsa

cbsa %>%
  select(NAME) %>%
  glimpse()
  

This data-set contains population estimates for CBSAs -- core-based statistical areas. What are these? 

> Metropolitan and Micropolitan Statistical Areas are collectively referred to as Core-Based Statistical Areas.
> Metropolitan statistical areas have at least one urbanized area of 50,000 or more population, plus adjacent territory that has a high degree of social and economic integration with the core as measured by commuting ties.
Micropolitan statistical areas are a new set of statistical areas that have at least one urban cluster of at least 10,000 but less than 50,000 population, plus adjacent territory that has a high degree of social and economic integration with the core as measured by commuting ties.
> Metropolitan and micropolitan statistical areas are defined in terms of whole counties or county equivalents, including the six New England states. As of June 6, 2003, there are 362 metropolitan statistical areas and 560 micropolitan statistical areas in the United States. [Source](https://www.census.gov/topics/housing/housing-patterns/about/core-based-statistical-areas.html)

Look at the column called `NAME` and note how It combines the state's name (abbreviated) and the name of the area, "Abilene, TX", "Callahan County, TX", "Jones County, TX", and so on. 

This is a common issue with many census data-sets and so it would be nice to be able to split up this `NAME` column into two pieces 

    (1) `placename` ("Abilene", "Callahan County", etc) and 
    (2) `stateabb` ("TX", "TX", etc.) 
    
We do this below, with the separation occurring where a `","` is seen in `NAME`. 

In [ ]:
cbsa %>%
  separate(
    col = NAME, 
    into = c("placename", "stateabb"),
    sep = ", ",
    remove = FALSE
  ) -> cbsa 

Let us see the original column and the two new columns we created.

In [ ]:
cbsa %>%
  select(NAME, placename, stateabb) %>%
  head()

Here is what each piece of code is doing:

| code     | what it does ... |
|:--       | :----            |
| col =    | identifies the column to be separated |
| into =   | creates the names for the new columns that will result |
| sep =    | indicates where the separation should occur |
| remove = | indicates whether the column to be separated should be removed from the data-set or retained once the new columns have been created. Setting it equal to `FALSE` will keep the original column, `TRUE` will remove it. |

What if the column to be separated was made up of numbers rather than text? Take the `STCOU` column, for instance. Look like numbers, right? Except these numbers are really identifiers, FIPS codes to be exact, with the first two digits flagging what state this area is in and the next three digits identifying the area itself. Ohio's FIPS code is, for instance, `39`, and Portage County's FIPS code is `133`. So here, it would be nice to create two new columns, one with the state FIPS code (`stfips`) and the second with the county FIPS code (`coufips`). We do this below, but this time setting `sep = 2` because we want the separation to happen after the second digit. 

In [ ]:
cbsa %>%
  separate(
    col = STCOU, 
    into = c("stfips", "coufips"),
    sep = 2,
    remove = FALSE
  ) -> cbsa

cbsa %>%
  select(STCOU, stfips, coufips) %>%
  head()

## unite()
This is the very opposite of `separate()` -- two or more columns are united into ONE column. For example, take the file I am reading in as `coudf`. This file has similar content to what we read in for the CBSAs but this one has data for counties and states. 

In [ ]:
read_csv(
  "https://www2.census.gov/programs-surveys/popest/datasets/2010-2018/counties/totals/co-est2018-alldata.csv"
  ) -> coudf

Let us filter this file so that we retain rows only for counties. I am doing this with `filter(COUNTY != "000")` because the state rows are the ones with `COUNTY == "000"`. 

In [ ]:
coudf %>%
  filter(COUNTY != "000") -> coudf2

coudf2 %>%
  select(STNAME, CTYNAME) %>%
  head()

Now I want to combine the county name (`CTYNAME`) and the state name (`STNAME`) into a single column, with the two names separated by a comma and a single white-space, i.e., by `", "`. 

In [ ]:
coudf2 %>%
  unite(
    col = "countystate",
    c("CTYNAME", "STNAME"),
    sep = ", ",
    remove = FALSE
  ) -> coudf2

coudf2 %>%
  select(CTYNAME, STNAME, countystate) %>%
  head()

Key elements here, are, 

| code | what it does ... |
| :--  | :--              |
| col = | identifies the `new column` to be created |
| c("..") | identifies the columns to be combined, as in c("column1", "column2", "column3") |
| sep = | indicates if we want the merged elements to be separated in some manner. Here we are using ", " to separate with a comma followed by a single white-space. But we could have used any separator or no separator at all |
| remove = | indicates if we want the original columns deleted `(TRUE)` or not `(FALSE)` |

## Pivoting data 
If I look at the original CBSA file `cbsa`, I see that it has been setup very oddly. In particular, starting with column 6 we have a jumble of information ...

+ CENSUS2010POP, ESTIMATESBASE2010, POPESTIMATE2010 all have the population totals for 2010. 
+ POPESTIMATE2011 through POPESTIMATE2018 are the population totals for 2011-2018 
+ NPOPCHG_20XY give us net population change for 2010-2018 
+ BIRTHS20XY give us the number of births for 2010-2018 
+ DEATHS20XY give us the number of deaths for 2010-2018 
+ NATURALINC20XY gives us the natural increase = births - deaths for 2010-2018 
+ INTERNATIONALMIG20XY gives us international immigrant totals for for 2010-2018 
+ DOMESTICMIG20XY gives us domestic migrant totals for 2010-2018 
+ NETMIG20XY give us net migration totals for 2010-2018 
+ RESIDUAL20XY gives us some small numbers left over after adding and subtracting the 

Let us keep only a few columns (columns 4, and then 8 through 16) to see what the current layout looks like.

In [ ]:
read_csv(
  "https://www2.census.gov/programs-surveys/popest/datasets/2010-2018/metro/totals/cbsa-est2018-alldata.csv"
  ) -> cbsa

cbsa %>%
  select(c(4, 8:16)) -> cbsa01

cbsa01 %>%
  head()

### Wide-to-Long with pivot_longer() 
Why did they not setup the data in such a way that it had the following `tidy` structure? This would make a lot more sense rather than having each year be a column all its own. 

| NAME | YEAR | POPULATION |
| :--  | :--  | :--        |
| Abilene, TX | 2010 | 165583 |
| Abilene, TX | 2011 | 166616 |
| Abilene, TX | 2012 | 167447 |
| ....        | .... | ....   |
| Callahan County, TX | 2010 | 13513 |
| Callahan County, TX | 2011 | 13511 |
| Callahan County, TX | 2012 | 13488 |
| ....        | .... | ....   |

Well, we can easily create the proper structure of the data-set, as shown below

In [ ]:
cbsa01 %>%
  group_by(NAME) %>%
  pivot_longer(
    names_to = "variable",
    values_to = "POPULATION",
    cols = 2:10
  ) -> cbsa01.long

cbsa01.long %>%
  head()

| code | what it does ... |
| :-- | :-- |
| names_to = | identifies the name of the new column that will be created |
| values_to = | identifies the name of the new column in which values will be stored |
| cols = | identifies the columns that will be pivoted from wide to long |
| group_by() = | holds unique combinations of whatever column names you put in `group_by()` fixed while it pivots the other columns |

I still need to clean up the variable column so that it only shows the four-digit year rather than POPESTIMATE2010, and so on. Let us do this next. 

In [ ]:
cbsa01.long %>%
  separate(
    col = variable,
    into = c("todiscard", "toyear"),
    sep = 11,
    remove = TRUE) -> cbsa01.long2

cbsa01.long2 %>%
  mutate(YEAR = as.numeric(toyear)) %>%
    select(c(NAME, YEAR, POPULATION)) -> cbsa01.long3

cbsa01.long3 %>%
  head()

Another way to do this would have been with the `stringr` package, something we will see a little of down the road. Here, we use a specific function (_str_remove_all_) from `stringr` to eliminate the "POPESTIMATE" string more efficiently.

In [ ]:
cbsa01.long %>%
    mutate(
    Year = stringr::str_remove_all(variable, "POPESTIMATE")
    ) -> cbsa.long4

cbsa.long4 %>%
    select(-variable) %>%
    head()

### Long-to-wide 

Now, let us say the data-set was a different one, perhaps the one shown below. This data-set comes from the 2017 American Community Survey and along with state FIPS codes (`geoid`) and state name (`NAME`) it contains data on `income` = median yearly income, `rent` = median monthly rent, and `moe` = the margin of error at the 90% confidence level.

In [ ]:
us_rent_income %>%
  head()

Notice here the setup looks weird because two different variables (_income_ and _rent_) have been combined in a single column. In addition, the estimate and the moe (the margin of error) for both income and rent is also squeezed into single columns respectively. 

Instead, the data-set should have been `tidyly` setup as follows:

| GEOID | NAME    | income | rent | moe_income | moe_rent |
| :--   | :--     | --:    | --:  | --:        | --:      |
| 01    | Alabama | 24476  | 747  | 136        | 3        |
| 02    | Alaska  | 32940  | 1200 | 508        | 13       |
| 03    | Arizona | 27517  | 972  | 148        | 4        |
| ...   | ...     | ...    | ...  | ...        | ...      |

Well, this can be done as well, with the `pivot_wider()` function that takes from the "long" format to the "wide" format. 

In [ ]:
us_rent_income %>%
  group_by(GEOID, NAME) %>%
  pivot_wider(
    names_from = variable,
    values_from = c(estimate, moe)
  ) -> usri.wide

usri.wide %>%
  head()

| code | what it does ... |
| :-- | :-- |
| names_from = | identifies the column from which unique values will be taken to create the names of the new columns that will result |
| values_from = | identifies the column(s) from which the values should be assigned to the new columns that will result |
| group_by() | holds unique value combinations of whatever column names you put in `group_by()` fixed while it pivots the rows to new columns |

### The example that follows is a tricky one so be careful!!  

Here is another example, this time with the `cbsa` data but showing you how we might use a combination of `pivot_longer()` and `pivot_wider()` 

In [ ]:
cbsa %>%
  select(3:5, 8:88) %>%
  group_by(NAME) %>%
  pivot_longer(
    names_to = "variable",
    values_to = "estimate",
    cols = 4:84
    ) -> cbsa.01

cbsa.01 %>%
  head()

Now I will clean up the contents of cbsa.01 so that year is a separate column.

In [ ]:
cbsa.01 %>%
  separate(
    col = "variable",
    into = c("vartype", "year"),
    sep = "(?=[[:digit:]])",
    extra = "merge",
    remove = FALSE
  ) -> cbsa.02

cbsa.02 %>%
  head()

Here, notice that the `sep = ` command looks different. What is this specification here? Well, if you look at the column names we need to flip in __cbsa__, you will notice that the four-digit year does not always start at the same place. Here are the column names ...

In [ ]:
names(cbsa)

Notice POPESTIMATE, NPOPCHG, BIRTHS, DEATHS, and so on. 

What we need therefore is to specify where the solumns should be separated, and we do that by saying separate wherever you see the first nuymeric digit appear. This is our `sep = "(?=[[:digit:]])"` command.

Now the final flip to wide format, by first omitting the first, third, and four columns, then grouping by NAME and year, and then pivoting wide ...

In [ ]:
cbsa.02 %>%
  select(c(2, 5:7)) %>%
  group_by(NAME, year) %>%
  pivot_wider(
    names_from = "vartype",
    values_from = "estimate"
  ) -> cbsa.03

cbsa.03 %>%
  glimpse()

And there you have it. We are just getting familiar with the heart of `dplyr` and `tidyr`, both written to work well together, and an essential part of any data analysts' toolkit. 

------

# Exercises for Practice

## Exercise 01 
Using the `cmhflights` data from last week, create a column that unites the three columns `Year`, `Month`, and `DayofMonth` into a single column that we will name `date_of_flight`. This column should separate the three fields by "-". 

> Hint: You will have to use `load(...)` with `here(...)`.

## Exercise 02
Sticking with `cmhflights`, separate `OriginCityName` into two new columns, `origin_city` and `origin_state`. Do the same for `DestCityname`, calling the new columns `destination_city` and `destination_state`, respectively. Both city columns should only display the name of the city, while both state columns should only display the abbreviated state name (for example, "CA", "OH", etc.)  

## Exercise 03
Tidy the `weather` data such that the resulting data-set, called `wdf`, has the `days` (the d1-d31 columns) as rows and `TMIN` and `TMAX` as columns. The end result should be as shown below:

| id  | year | month | day | TMIN | TMAX |
| :-- | :--  | :--   | :-- | :--  | :--  |
| MX000017004 | 2010 | 1 | d1 | NA | NA |
| MX000017004 | 2010 | 1 | d2 | NA | NA |
| MX000017004 | 2010 | 1 | d3 | NA | NA |
| MX000017004 | 2010 | 1 | d4 | NA | NA |

The data can be accessed as shown below. 

In [ ]:
read.delim(
 file = "http://stat405.had.co.nz/data/weather.txt",
 stringsAsFactors = FALSE
 ) -> weather

weather